In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

from selenium.webdriver.common.keys import Keys

In [2]:
name=['헤오드']
category=['별점']

# 헤오드 디퓨저 후기
ns_address="https://search.shopping.naver.com/catalog/28972880334?query=%EB%94%94%ED%93%A8%EC%A0%80&NaPm=ct%3Dlb3uasc0%7Cci%3D12c5395c1222085e1399a68267960a6b360b600e%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D14a3745427931bf1d00aa2fd471fdc645c88b903"
#xpath
shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[1]/ul/li[2]/a"
category1="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[2]/div[2]/ul/li[2]/a" #5점
category2="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[2]/div[2]/ul/li[3]/a" #4점
category3="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[2]/div[2]/ul/li[4]/a" #3점
category4="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[2]/div[2]/ul/li[5]/a" #2점
category5="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[2]/div[2]/ul/li[6]/a" #1점

In [3]:
#함수 선언
def add_dataframe(name,category,reviews,stars,dates,options,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star','date','option'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i],dates[i],options[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,date,option,'null','null']
        n+=1    
    return df1

In [4]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_4553/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
# 5점 리뷰 가져오기
d.find_element(By.XPATH,category1).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

1 플라워마켓 우드랜드 리틀포레스트 그린허브 이렇게 4가지향 구매했어요 우드랜드는 남자스킨향 같은데 너무 시원하고 좋은 향이예요 화장실에두니 상쾌하네요 스틱하나만으로 충분해요 개인적으로 제가사용하는 샴푸향과 동일해서 더 맘에 들어요

리틀포레스트는 서재에 어울리는향이예요 방에들어갈때마다 느껴지는 청량감이 좋으네요 화장실보다는 넓어서인지 은은한 발향이 좋아요

플라워마켓은 달달한 향은 아니예요 상품설명처럼 꽃가게 들어갔을때 향과 동일한인데 침대 옆 협탁에 두니 자려고 누웠을때 느껴지는 향이좋아요 단지 침대에 누웠을때만 느껴질정도로 향이 세진 않아요

그린허브는 제가좋아하는 향수와 향이 많이 비슷해요
제일 좋아하는 향이라 애들 침실에 뒀는데 애들이 좋은 향기가 난다고 좋아합니다 단지 발향은 네가지중 제일 약하네요 하루밖에 안되서 인걸까요 ?

발향은 우드랜드 리틀포레스트 플라워마켓 그린허브 순으로 센것 같고요 아무래도 공간크기 영향도 있을것 같네요 거실같은 큰공간은 200미리 두개정도는 필요하지 않을까 싶어요

거실과 주방엔 디퓨저를 놓지않았는데요, 덧붙이자면 발향이 약해서 향이 느껴지지않는다고 해도 거실에서 방으로 들어갔을때의 공기질은 다르긴합니다

길게쓰는거 귀찮아서 안좋아하는데 어제 뜯어보고 너무좋아서 쓰다보니 길어졌네요 그만큼좋고 재구매의사도 있어요~~ 평점5 21.11.18. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스 

2 작은 공방이 환기를 시키고 문닫아 놓으면 새건물인데도 가구냄새와 함께 탁한 공기 냄새? 때문에 불쾌해서 캔들과 디퓨저로 고민하다 주문해봤어요
집에서도 사용하려고 주문하며 향이 궁금해 세가지와 서비스까지 4종 다양하게 골라봤는데 전체 다 코 아프게 강하지 않아 만족하며 가장 좋아하는걸로 공방 한쪽에 놓아봤는데 자주 오던 분이 문 열고 들어서자마자 바로 알아차리고는 너무 좋다고 해주시네요
순간적으로 강하게 뿌려지는게 아닌 은은하게 퍼지는 잔향 같은 느낌이라 공기가 맑아진 기분이라고

17 -리틀포레스트
진짜 상쾌한 풀냄새 나요! 바람에 실려오는 풀냄새st. 방에서 공부하는데 향이 은은하면서도 상쾌해서 집중도 잘됩니다.

-블랙체리
이건 너무나 유명한 향이라서 기대 안했는데 시중에 싸게 파는 향이랑 달라요! 약간 더 고급느낌입니다 너무 맘에 들어요

- 라벤더
라벤더 향도 저렴한 향 안나고 좋아요! 잘못하면 약간 짠내 비슷한 게 나는데 여기 라벤더는 그런거 전혀 없네요

나머지 하나는 아직 안뜯어봤는데 그것도 기대되네여 ㅎㅎ 여기 디퓨저가 유명한 이유가 있네요! 3개 다 맘에 듭니다! 디퓨저에 꽂는 풀도 너무 귀엽고ㅠ 인테리어 기능까지… 향기도 좋은데 비주얼로도 기분이 좋아져요! 이제 디퓨저는 여기서 재구매할 것 같아요 평점5 22.03.05. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 미니 몬스테라 

18 사은품으로 고른 블랙체리 너무 너무 좋아욤! 욕실에서 반짝반짝 다이내믹하게 욕실의 무드를 리드하고 있습니당! 달콤한 글래머러스 향 너무 너무 맛있는 센스 감각 sense 향이에욤! 매우 로맨틱한 캔디 같은 페미닌하면서도 발랄함과 세련된 프루티함을 지녔으며, 포근포근하여 눈을 살짝 감고 깊게 로맨스를 숨쉬어욤! 달콤함 프루티함과 여성스러움 페미닌의 힘을 극대화 하고 있으며 황금기의 시간과 공간성을 보여주며, 압권입니당! Pure 함! 큐트함! 러블리 러블리 사랑스러움이 압도합니당! 낭만성! 판타스틱 해욤! 엄청 엄청 아름다움! 엄청 엄청 훌륭함! 해욤! 아! 너무 달콤해욤! 헤오드! 너무 너무 멋져욤! 영원한 황금기의 로맨티시즘! 러블리 러블리 해욤! 그리고 퀸 queen 의 매우 완전한 리더십 과 people 과 국가 nation 를 이룬, 을, 보여주고 있습니당! 엄청 엄청 완벽함! 해욤! ! 엄마랑 향 같이 고를 때 ! 헤오드 그룹!, 짱! 완벽함 Perfection! 언제나 깊고 많은 감사함을 드립니당! 많이 많이 감사드려욤! 배송도 빠르고 포장도 완벽해욤! *^_____________^* 평점5

33 일단 상품 상세페이지에 액체 색상이 투명해서 투명인줄 알고 주문했는데 노~오~란 색 액체(미스티 포레스트)가 와서 변색인줄 알고 문의했더니 향마다 색상이 다르다고 하는군요. 원하는 투명색이 아니라서 실망을 했지만 상세한 답변과 전화까지 주셔서 설명해 주셔서 마음이 좀 풀렸(?)습니다
향별로 액체의 색상도 상세설명이 안내해주셔야 저처럼 헷갈리는 사람이 없지 않을까 싶네요.
게다가 200ml 상세샷에 있는 포장 상자의 사진과 제가 배송받은게 달라서 물어보니 이것도 지난주부터 바뀌었다며 상세사진 교체예정이라고합니다. 저야 제가 사용할거니 박스가 별 상관은 없었지만 선물용으로 구매하신 분들은 참고하셔야할 것 같습니다. 이번에 바뀐 상자는 내용물이 너무 섞여 있어서 그대로 선물하면 좀 안될것 같은...
120ml는 이미 꺼내서 사용해봤는데 이건 다음달에 이사가서 개봉하려고 아직 사용전이라 향이 어떤지는 잘 모르겠습니다.
병이 좀 큰 감이 있고요 120ml와 비교했을 때 가격이 두 배인건 생각하면 200ml 한개 살 가격으로 120ml 2개 살 수 있으니까 상대적으로 가성비는 좀 아쉬운거 같아요. 평점5 20.09.22. 디퓨저 선택: 천연 디퓨저 200ml / 향 선택: 미스티 포레스트 

34 디퓨저를 이곳저곳에서 많이 사봤는데 르몽드가 제일 발향력이나 인테리어에 최고인거같아요.
간혹 다른 디퓨저 사면 싸구려향같은거나 머리아픈적이 많았는데 여기는 유해성분도 없고 향도 고급지게 느껴지고 머리아프지 않고 계속 맡고 싶은 향이예요!!!
지인한테도 추천한다고 소문내고있어요. 특히나 라탄이랑 너무 찰떡이예요 여러분 라탄 필수입니다!

포장상태도 에어캡으로 잘 감싸져있고 더 흔들리지 않게 신문지로도 더 해주시는거같아요.
포장에 감동.. 디퓨저 케이스도 깔끔하고 안에 디퓨저도 흔들리지 않게 단단한 속지도 들어있고 사용방법도 같이 동봉해주셔서 좋았어요! 처음사용하시는분들이나 처음에 섬유스틱을 4개 바로 사용하시는 분들이 계실텐데 사용방법에 스틱1개 먼저 사용해보고 기호에 따라

46 제품이 예뻐보여서 구입했는데요. 인테리어 효과도 좋고 너무 만족스러워요! 집하고 사무실에 놓고 쓰려고 구입했는데 어떤 향을 골라야 좋을지 몰라 집에서는 늘 쓰던 무난한 블랙체리를 놓아봤어요.

처음 알루미늄 제거할 때 제가 요령이 없는지 잘 안떨어지더라고요. 뜯는데 좀 힘들었어요. 그리고 사용설명서에 처음에는 섬유스틱 하나만 꽂으라고 했는데 전 향이 별로 안나는 것 같아서 2개 꽂았다가 나중에 하나 뺐어요. 꼭!! 첨에 하나만 꽂아두세요!!! ^^;; 시간지나니 은은히 향이 잘 퍼지더라고요. 하나만 꽂으시라는 이유가 다 있었다는... 향 잘 납니다. 용량이 작아서 향이 너무 안나면 어쩌나했는데 괜찮아요. 저는 향이 은은하게 퍼지는 게 좋거든요. 다른 제품 구입한 것도 궁금해요. 그건 아예 제가 써보지 않은 향이라 기대됩니다. 뜯으면 후기 남길게요! 평점5 21.10.22. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 안개꽃(핑크) 

47 리뷰를 참고하긴 하지만 개인취향이 다 달라 믿고 샀다가 향이 맘에 들지않기도, 에탄올 냄새만 강해 설명과 다름에 반품시킨적도 있어서, 이후로 매장에서 직접 향을 체크하고 구입했던 디퓨저가 다 되어 갑작스레 온라인 구매를 알아보던중 이번 제품역시 속는셈치고 리뷰를 꼼꼼히 살펴보고 구입하게 되었습니다.

사실 리뷰 남기는거 좋아하지않는데, 제가 써 보고 좋으면 저와같은분이 참고하셨으면 해서 올리는것이니 참고만하세요~^^

향은 그린허브, 프레쉬 레몬,블루밍자스민,가든스윗피
이렇게 4개를 주문했고 +1 서비스로 그린허브가 1병 더 왔습니다.

그린허브는 침실에 놓고 프레쉬레몬은 욕실에
블루밍자스민은 현관쪽에 배치했는데~
아직 사용하지 않았고 다른 향들은 구입한적없어 모르지만, 현재 구입한 그린허브, 프레쉬레몬,블루밍자스민은 재구매 의사가있는만큼 괜찮네요.^^ 평점5 22.08.12. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 프레쉬 레몬 / 식물 데코: 미니 몬스테라 

48 온라인에서 디

69 뱌송 빠르고 병도 예쁘고 깔끔하네요. 꽃집에서 나는 향 기대했는데 엄청 비슷하진 않아도 그래도 싱그러운 느낌이라 좋아요. 처음에는 아무 냄새 안나는 것 같더니 시간 지나니 은은하게 퍼지네요.
쿠*에 배송도 빠르고 가격도 저렴한 디퓨저들 진짜 많지만 일부러 더 비싸고 기다려가며 여기서 산 이유는 그만큼 발향이 좋고 오래 가길 바라는 마음으로 산거예요. 한달 뒤에 또 리뷰 남길게요. 평점5 21.01.07. 디퓨저 선택: 화이트 디퓨저 120ml / 향: 플라워 가든 / 식물 데코: 미니 몬스테라 

70 은은한 향보다는 진하게 나는 디퓨저를 개인적으로 선호합니다.
시간 지나면 발향이 낮아져서 향이 있는지 없는지 모를 정도가 되거든요. 그런 점도 있고 또 제가 비염이 있는지라 냄새를 잘 맡지 못합니다.
그래서 방향 좋은걸 찾았는데 여기 제품이 제 취향에 딱 맞는거 같아요.
패키지도 진짜 깔끔하고 어디에서든 다 잘어울려요.
가족들도 제 방에 설치한 디퓨저 사달라고 하더라구요.
한달 정도 써보고 괜찮으면 또 구매하겠습니다. 평점5 21.03.23. 디퓨저 선택: 화이트 디퓨저 200ml / 향: 블랙 체리 / 식물 데코: 카라(핑크) 2ea 

71 사무실과 가정에서 사용중입니다. 자사의 설명데로 농축된 향이 잘 풍겨서 발향에 대한 부분은 이루말할것 없이 좋습니다.
다만, 평소에 옅은 발향과 향기를 좋아하시는 분들은 조금 어지렵거나 좀많이 진하다는 느낌을 많이 받으실겁니다. 제가 처음에 그랬거든요.
그런데 몇일 지나고 집에 들어가면 좋은 향기가 은은하게 사무실에 쾌쾌한 냄새가 연해지면서 자리에서 좋은 향기들이 적당히 발향되고 나서부터는 만족감이 올라갑니다. 평점5 21.12.31. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 라그라스(핑크) 

72 배송이 빠르고 포장도 에어캡에 쌓여서 꼼꼼하게 잘 되어 왔어요.
깔끔하고 예쁜 디자인에 향기까지 좋아 마음에 쏙 들어요~^^
투명한 병에 로즈데코까지 러블리하고 예뻐서 인테리어 효

...

1264 여기서 계속 시키고있어요 가장 맘에드는 향은 가든스윗피 블랙체리 에이프릴플라워에요 평점5 22.02.15. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 미니 몬스테라 

1265 너무 좋아요 믿고구매하는곳이에요 또 다른것도 사러올께요 이쁜거 많이 부탁드립니다~^^ 평점5 21.06.23. 디퓨저 선택: 리필 용액 110ml / 향: 우드 랜드 / 식물 데코: 식물 미포함 

1266 기대한 빨래 냄새는 아니지만 향 너무 좋아요. 장식으로 온 가짜 식물 꽂이(?)는 별로예요. 평점5 22.04.16. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 안개꽃 (아이보리) 

1267 리틀포레스트 화장실에 뒀는데 향 너무 세지도 않고 시원한 느낌이 좋네요
그리고 예뻐요~ 평점5 22.03.11. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 리틀 포레스트 / 식물 데코: 미니 몬스테라 

1268 일단 발향도 너무 잘되고 냄새도 너무 좋아요 침실에 뒀는데 같이 사는 사람이 향에 좀 민감한데 향이 괜찮은지 무슨 향이냐고 먼저 물어보더라구요ㅎㅎ 일단 잎도 너무 파릇파릇하니 인테리어로도 너무 좋아요~ 평점5 21.02.20. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 플라워 L 

1269 사무실용으로 은은하게 향도 좋고 안에 설명서가 있어서 궁금증을 바로 해결해서 좋아요! 평점5 21.01.29. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 유칼립투스 

1270 처음에는 향이 좀 찐한가 생각했는데 지날수록 은은해 지네요. 풀냄새랑 비슷한거 같아요 평점5 21.10.16. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스 

1271 브라운색 병으로 구매하길 잘한거 같아요~ 인테리어하기도 좋고 향도 은은해서 넘 좋네요 평점5 22.05.05. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 가든

1328 맘에들어요 ㅎㅎ 다만 몬스테라는 넘 싼티나고 하나는 털 엄청 날려요 ㅠ 그래듀 이뻐요 평점5 20.10.31. 디퓨저 선택: 브라운 천연 디퓨저 120ml / 향 선택: 에이프릴 프레쉬 

1329 생각보다 향이 강하네요. 향도 좋아요~ 사무실에 놔뒀는데 한 10평되는곳에 전부 꽃향기네요.ㅎㅎ 나중에 다 쓰면 추가 구매 해야겠네요~ 평점5 22.07.08. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 안개꽃 (아이보리) 

1330 은은하고 자연을 담은 향기♥ 인공적이지 않아 좋아요. 각방에 하나씩 놓았어요😄 평점5 21.04.01. 디퓨저 선택: 화이트 디퓨저 200ml / 향: 플라워 가든 / 식물 데코: 튤립(오렌지) 2ea 

1331 그린허브 향 너무 좋아요! 탈의실에 뒀는데 들어갈때 마다 기분좋고 상쾌하고 향긋해요! 평점5 21.06.21. 디퓨저 선택: 리필 용액 110ml / 향: 그린 허브 / 식물 데코: 식물 미포함 

페이지 86 

1332 예쁘고 향은 좋으나 매장에 두기에는 발향이 약한 편이예요..!! 여러개 둬야할 것 같아요 평점5 22.08.16. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 플라워 L 

1333 문옆에 두니 오며가며 적당히 향 뿜뿜해주네요
모형식물은 조잡해요..없어도 될듯합니다 평점5 22.08.23. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 유칼립투스 

1334 너무 맘에들어용 이곳저곳
올려놔도 잘어울리고 예쁘네요 ㅎㅎ
진짜 잘산거같아용 😍 평점5 21.11.10. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 안개꽃(핑크) 

1335 계속해서 블랙체리향으로만 사용하고 있어요 너무 은은하니 좋고 독하지 않아서 좋아요 평점5 22.02.07. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 안개꽃(핑크) 

1336 향이 은은하고 

1393 생일때 친구가 선물로 보내줘서 쓰다가 이번에 새로 주문했는데 역시 향 대박이에요.. 평점5 21.05.03. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 우디 포레스트 / 식물 데코: 유칼립투스 

1394 생각했던 향은 아니지만
그래도 이것도 나름 좋네요
쓰던거 다사용하고 쓸 예정입니다 평점5 20.09.19. 디퓨저 선택: 천연 디퓨저 120ml / 향 선택: 미스티 포레스트 

페이지 90 

1395 스프링데이는 복숭아 향기라고 나와있어서 샀는데 전혀 복숭아 향기 아니에요 ㅠㅠ 그냥 꽃향기..? 그래서 화장실에 냅뒀어요~
블랙체리는 최고 👍👍
침대맡에 두니 좋네용 평점5 21.10.01. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 유칼립투스 

1396 두가지를 사서 이건 향을 맡아보지는 못했지만
첫번째 제품에 너무 만족해서 충분히 훌륭할것으로 예상되네요 이 향도 만족 스러우면 걱정이네요 두가지를 동시에 향을 할 수는 없으니 ㅡ.ㅡ 평점5 22.08.13. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 리틀 포레스트 / 식물 데코: 유칼립투스 

1397 작아도발향좋구요 향이일단 다 거부감없이은은해서좋았어요 브라운병도 나중에구매해봐야겠어요 화이트병은깔끔해서이쁘네요 다른향들도궁금해서 다른것도 구매해보고 싶어요 평점5 20.10.17. 디퓨저 선택: 화이트 천연 디퓨저 120ml / 향 선택: 하와이안 라임 비치 

페이지 91 

1398 향 진하고 좋아요!! 꽃도 갬성있습니당! 다만 스틱을 처음부터 너무 많이 사용하면 금방 날아가니까 하나씩 사용하시길 추천드립니다! 평점5 21.08.01. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 프레쉬 레몬 / 식물 데코: 미스티블루 

1399 이사오면서 냄새가 안빠져서 주문했는데 잡내 다 잡아주고 향까지 너무 좋아요ㅎㅎ집에 들어오면 기분이좋네요 평점5 20.07.20. 디퓨저 선택: 화이트 디퓨저 120ml / 향: 에이프릴 프레쉬 / 식물 데코: 미니 몬스테라

페이지 95 

페이지 96 

1459 정말 여러 디퓨저를 많이 써보고 있는데
이번에 산 이 디퓨저는 우선 디자인부터 취저!!!!
향도 잘 나고 맘에 듭니당 ^.^ 다양한 향으로
여러개 구입했습니다 평점5 20.07.28. 디퓨저 선택: 천연 디퓨저 120ml / 향 선택: 힐링 라벤더 

1460 배송도 빠르고~ 합리적인 가격에 향도 넘 좋아요~ 화장실에 두었는데 넘 이쁘네요~~ 평점5 22.08.09. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 유칼립투스 

1461 디퓨저에 라탄받침 꽃까지 꽂아놓으니 냄새는 물론 인테리어 소품으로도 딱입니다 평점5 21.07.14. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 유칼립투스 

1462 화장실에 두려고샀어요
밑에 라탄받침도 같이 놓으니 훨씬이뻐요
탄탄하니 좋아요 평점5 21.01.02. 디퓨저 선택: 화이트 디퓨저 120ml / 향: 에이프릴 프레쉬 / 식물 데코: 목화블렌드 

1463 생각보다 발향이 잘 되어서 너무 좋았어요!
하나만 꽂아도 충분하게 느껴졌습니다:) 평점5 21.09.25. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스 

1464 더 큰용량도 있으면 좋겠어요
받자마자 뜯어서 올려놨는데 성분착해서 맘에듭니다. 평점5 22.05.24. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 미니 몬스테라 

1465 기존에 잘 사용해서 리필액 추가로 구입했어요 ! 가든 스윗 피 향 너무 좋아요 ㅎㅎ 평점5 21.08.06. 디퓨저 선택: 리필 용액 200ml / 향: 가든 스윗 피 / 식물 데코: 식물 미포함 

1466 향이 너무좋아요 디자인적으로도 보면 이쁘구요 화장실 냄새 잘 잡아주는것같아여 평점5 21.11.15. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 우드 랜드 / 식물 데코: 미스티블루 

1467 향이 좋네요 다른향도 구매해봐

1524 이 디퓨더를 쓴후 부터 다른 디퓨저는 눈에 안들어옵니다 향도 좋고 오래유지되요 집에꽃향기가 나서 좋아요^^ 평점5 22.11.21. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 안개꽃 (아이보리) 

1525 하나는 화장실에 두려고 레몬? 으로 샀어요! 쾌쾌한 냄새가 싫어서 상큼한 향으로 골랐는데 이것도 만족합니당 평점5 22.01.11. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 프레쉬 레몬 / 식물 데코: 플라워 S 

1526 배송도 빠르고 포장도 꼼꼼하네요. 향도 예상한 것과 같이 좋았고 식물도 예뻐요 평점5 22.02.23. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 유칼립투스 

1527 향이 오래 가서 너무 좋았어요 :)
벌써 다 써가네요 ㅎㅎ
재구매 각입니다😆🤍 평점5 21.08.01. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 미니 몬스테라 

1528 향이 은은하고 식물과함께 데코하니 넘 예뻐요^^
스틱은 하나만해도 충분하네요^^ 평점5 21.01.07. 디퓨저 선택: 화이트 디퓨저 120ml / 향: 가든 스윗 피 / 식물 데코: 미니 몬스테라 

1529 배송 빠르고 향이 좋아요 집 분위기랑 어울리고 좋은 선택을 해서 기분이 좋네요 평점5 22.08.05. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 그린 허브 (BEST 3) / 식물 데코: 미니 몬스테라 

1530 온 집안이 꽃향기로 가득해졌네요~
향도 잘퍼지고, 인테리어 효과까지..
굿입니다. 평점5 21.06.16. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 가든 / 식물 데코: 플라워 L 

1531 다른향보다 레몬향이 빨리 닳는 느낌이지만 그래서 큰사이즈로 재구매할겁니다! 평점5 22.04.24. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 프레쉬 레몬 / 식물 데코: 미니 몬스테라 

1532 포장 굳 ^^ 향기도 알코올

In [6]:
# 5점 리뷰 데이터프레임으로 표현
df1=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df1.head()

,type,category,review,star,date,option
1,헤오드,별점,플라워마켓 우드랜드 리틀포레스트 그린허브 이렇게 4가지향 구매했어요 우드랜드는 남자...,평점5,21.11.18.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스
2,헤오드,별점,작은 공방이 환기를 시키고 문닫아 놓으면 새건물인데도 가구냄새와 함께 탁한 공기 냄...,평점5,22.07.08.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스
3,헤오드,별점,향에 민감한 편이라 디퓨저를 구매할 때 후기 몇십개씩 보고 사는데\n가격도 너무 괜...,평점5,21.01.04.,디퓨저 선택: 화이트 디퓨저 120ml / 향: 그린 허브 / 식물 데코: 플라워 L
4,헤오드,별점,"플로리아, 라그라스 우왕! 매우 퀸 queen 처럼 우아하고 아름다우며 너무 너무 ...",평점5,22.10.04.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 플로리아 / 식물 데코: 플라워 T
5,헤오드,별점,오랜만에 다시 재구매했는데 그동안 많은 발전이 있었네요☺️\n센스가 어찌 이리 좋은...,평점5,21.08.12.,디퓨저 선택: 화이트 디퓨저 200ml / 향: 클린 코튼 / 식물 데코: 카라(핑...


In [7]:
# 크롤링한 5점 데이터의 갯수
len(df1)

1572

In [8]:
# 5점 데이터 전처리

df1 = df1.drop_duplicates()
df1 = df1.reset_index(drop=True)
df1_1 = df1['option'].str.split('/',expand=True)
df1 = pd.concat([df1,df1_1],axis=1)
df1.head()

,type,category,review,star,date,option,0,1,2
0,헤오드,별점,플라워마켓 우드랜드 리틀포레스트 그린허브 이렇게 4가지향 구매했어요 우드랜드는 남자...,평점5,21.11.18.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스,디퓨저 선택: 브라운 디퓨저 110ml,향: 그린 허브,식물 데코: 유칼립투스
1,헤오드,별점,작은 공방이 환기를 시키고 문닫아 놓으면 새건물인데도 가구냄새와 함께 탁한 공기 냄...,평점5,22.07.08.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스,디퓨저 선택: 화이트 디퓨저 110ml,향: 가든 스윗 피,식물 데코: 유칼립투스
2,헤오드,별점,향에 민감한 편이라 디퓨저를 구매할 때 후기 몇십개씩 보고 사는데\n가격도 너무 괜...,평점5,21.01.04.,디퓨저 선택: 화이트 디퓨저 120ml / 향: 그린 허브 / 식물 데코: 플라워 L,디퓨저 선택: 화이트 디퓨저 120ml,향: 그린 허브,식물 데코: 플라워 L
3,헤오드,별점,"플로리아, 라그라스 우왕! 매우 퀸 queen 처럼 우아하고 아름다우며 너무 너무 ...",평점5,22.10.04.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 플로리아 / 식물 데코: 플라워 T,디퓨저 선택: 화이트 디퓨저 110ml,향: 플로리아,식물 데코: 플라워 T
4,헤오드,별점,오랜만에 다시 재구매했는데 그동안 많은 발전이 있었네요☺️\n센스가 어찌 이리 좋은...,평점5,21.08.12.,디퓨저 선택: 화이트 디퓨저 200ml / 향: 클린 코튼 / 식물 데코: 카라(핑...,디퓨저 선택: 화이트 디퓨저 200ml,향: 클린 코튼,식물 데코: 카라(핑크) 2ea


In [9]:
# 전처리 후 크롤링한 5점 데이터의 갯수
len(df1)

1545

In [10]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_4553/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [11]:
# 4점 리뷰 가져오기
d.find_element(By.XPATH,category2).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

1 플라워마켓 우드랜드 리틀포레스트 그린허브 이렇게 4가지향 구매했어요 우드랜드는 남자스킨향 같은데 너무 시원하고 좋은 향이예요 화장실에두니 상쾌하네요 스틱하나만으로 충분해요 개인적으로 제가사용하는 샴푸향과 동일해서 더 맘에 들어요

리틀포레스트는 서재에 어울리는향이예요 방에들어갈때마다 느껴지는 청량감이 좋으네요 화장실보다는 넓어서인지 은은한 발향이 좋아요

플라워마켓은 달달한 향은 아니예요 상품설명처럼 꽃가게 들어갔을때 향과 동일한인데 침대 옆 협탁에 두니 자려고 누웠을때 느껴지는 향이좋아요 단지 침대에 누웠을때만 느껴질정도로 향이 세진 않아요

그린허브는 제가좋아하는 향수와 향이 많이 비슷해요
제일 좋아하는 향이라 애들 침실에 뒀는데 애들이 좋은 향기가 난다고 좋아합니다 단지 발향은 네가지중 제일 약하네요 하루밖에 안되서 인걸까요 ?

발향은 우드랜드 리틀포레스트 플라워마켓 그린허브 순으로 센것 같고요 아무래도 공간크기 영향도 있을것 같네요 거실같은 큰공간은 200미리 두개정도는 필요하지 않을까 싶어요

거실과 주방엔 디퓨저를 놓지않았는데요, 덧붙이자면 발향이 약해서 향이 느껴지지않는다고 해도 거실에서 방으로 들어갔을때의 공기질은 다르긴합니다

길게쓰는거 귀찮아서 안좋아하는데 어제 뜯어보고 너무좋아서 쓰다보니 길어졌네요 그만큼좋고 재구매의사도 있어요~~ 평점5 21.10.22. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 가든 / 식물 데코: 유칼립투스 

2 후기에 클린코튼향이 좋다는게 많은거같아서 한가지 향으로 통일했는데 향은 은은하니 좋아요~ 설명서가 첨부되어있어서 도움이 됐어요~~ 첫날이라 스틱 한개만 꽂았는데 첫날 너무 강할까봐 걱정했는데 방에 놔도 냄사가 강하지않고 괜찮네요~ 이대로 지속기간만 진짜 길다면 성분도 안전하다 하니 좋은거같아요~^^
한가지 아쉬운건 120ml가 생각보다 진짜 작아요;;; 갈색병도 200ml짜리 있음 큰걸로 하고팠는데 왜 갈색병은 큰사이즈 없는건가요?ㅠㅠ 추가해주세요~~투명한 병은 흔해서 전 갈

33 처음 마개 땄을 때 향이 진해서 걱정했는데 스틱 꽂고 꽃 꽂고 나니 적당하게 향이 나네요. 옷방, 주방, 화장실 이렇게 두려고 샀어요. 조금 더 써봐야겠지만 우선 발향 잘 되고 은은하면서 향도 좋아요. 디퓨져 쓰면 알콜 냄새처럼 뭐가 독하게 나는 것도 있어서 머리 아프기도 하고 그런데 전혀 그런게 없네요! 평점4 21.11.17. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 안개꽃(핑크) 

34 디퓨저를 사용하자마자 색이 변하는 현상을 겪어서 문의드렸는데 판매하는 제품에 대해 책임감이 대단하다 느꼈어요! 새제품 보내주신 것도 같은 현상을 겪었는데 풀잎 데코가 문제였던걸류 결론이 나왔어요:’) 지금은 문제를 싹 고쳤을것이라 생각해요! 재주문 할 예정이에요 잘 부탁드립니다♥️ 평점4 20.09.10. 디퓨저 선택: 리필 용액 120ml / 향 선택: 블랙 체리 

35 병도 깔끔하고 향도 좋아요^-^
거실 화장실이랑 안방화장실에 뒀어요
다른 방들은 타 디퓨저 사용 중인데..비교한번 해봐야겠어요
사이즈가 아담해서 빨리 닳겠지만~다 쓰고 다른향도 구매해서 써보고싶네요~ 근데..프리저브드 식물 하나 꽃잎 모양이 이상해서.. 그냥 잘라버렸어요ㅠ 속상하네요.. 금액 추가해서 구매한건데.. 그래서 별하나 뺐어요 평점4 22.12.01. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 미니 몬스테라 

36 이향은 아직 오픈은하지않았어요! 우선 나머지 다쓰고 뜯을 예정이고요 3개중 2개가 제기준 성공이여서 이향또한 제 취향 적중할꺼라고 생각하고있습니다 나중에 쓰려고 고이 모셔뒀어요^^ 그리고 사은품방향제 넣어주신거는 차에다사용하면 찰떡일꺼같더라고요 잘사용할께요! 평점4 21.02.02. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 스프링 데이 / 식물 데코: 유칼립투스 

37 화장실에 비치하려고 구매했습니다. 포장 상태 이쁘고 배송도 빨라서 좋았습니다. 화장실용 검색해서 플라워 가든으로

71 어린이집 졸업선물로 선생님께 드리려고\n샀어요. 부담스럽지않는선에서\n고르다보니 선택했는데\n괜춘한거같아요.\n서비스로주신 방향제도\n향이좋아요.욕실에 걸어두었네요 평점4 21.02.19. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 스프링 데이 / 식물 데코: 미니 몬스테라 

72 플라워가든 다들 좋다고해서 삿는데
꼭 하나만 사서 향 맡아보시고 사세요!!! 저는 불호ㅠ
시향하는거 키트로 있었으면 좋겠어요 ㅜㅜ
디자인은 심플하고 좋아욧 평점4 21.06.21. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 가든 / 식물 데코: 미니 몬스테라 

73 향이 생각보다 강해요. 은은한 향은 아닌 것 같아요. 천연이라고 하니 믿고 써 볼게요. 튤립 조화는....너무 ㅆ..싼티가 나네용^_ㅠ 못 꽂아두고 진열장 행.. 평점4 20.10.20. 디퓨저 선택: 화이트 천연 디퓨저 200ml / 향 선택: 미스티 포레스트 

74 디자인 보고 구매했습니당
역시나 너무 깔끔하고 예쁘고
유칼립투스는 밝은 연두색인줄 알았는데
어두운초록색이네요
발향력이 끝내줍니다
다음엔 다른향 사보고싶어요 평점4 21.07.06. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 유칼립투스 

75 어린이집 졸업선물로 선생님께 드리려고
샀어요. 부담스럽지않는선에서
고르다보니 선택했는데
괜춘한거같아요.
서비스로주신 방향제도
향이좋아요.욕실에 걸어두었네요 평점4 21.02.19. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 클린 코튼 / 식물 데코: 유칼립투스 

76 화장실에 놓을 거라 향을 고민하다가 우드랜드가 발행이 좋다고 해서 골랐는데 역시나 만족스러워요. 가성비 너무 좋은데 디퓨저스틱이 4 개만 들어있어서 조금 아쉬워요 평점4 22.07.04. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 (BEST 3) / 식물 데코: 안개꽃 (아이보리) 

77 배송도빠르고 카페에서 전시되있는걸 보고 마음에들어서 구입하게되었습니다. 꽃향 약간

130 꾸준히 8개월정도 사용중이며 선물용까지 6번째 주문인데 전에는 본품 리필 관계없이 3+1 을 보내주셨는데.이번에는 본품 2개 리필 2개 시켰는데 사은품한개는 오지않았네요..저번이랑 바뀐 이벤트인건지..ㅡㅡ
그래서 별하나 뺐어요~쭉 써오고 주문하던지라..항상 사은품이 왔었는데 이번에 빠져서 좀 비싸단 생각이 들긴하네요..ㅡㅡ 평점4 22.02.05. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 마켓 / 식물 데코: 미니 몬스테라 

131 발향이 너무 좋은데 잘못맡으면 약간 약품냄새? 같은것도 나요 ㅠㅠ
가까이서 맡으면 엄청 상큼한 레몬향인데 ㅠㅠ
핑크뮬리에서는 왜 안씻긴 멍멍이 냄새가 나는거죠? ㅋㅋㅋ ㅠㅠ의아..
스틱 2개꽂았는데 엄청 빨리 닳아요.. 다음에는 200으로 시켜야겠어요 ㅠ 평점4 21.07.21. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 프레쉬 레몬 / 식물 데코: 라그라스(핑크) 

132 배송박스 다찌그러져서와서 돈주고 산물건인데 어디서 패대기 친거 받은 느낌이예요
이건 판매자 잘못은 아니죠ㅠ
향은 머리아프거나 속울렁거리진 않은데 아..디퓨져있었지..하는 은은하게퍼지는 향이네요 평점4 22.06.03. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 리틀 포레스트 (BEST 2) / 식물 데코: 안개꽃(핑크) 

133 블루밍자스민 향은 익숙하면서 괜찮네요. 리틀 포레스트는 기대했는데 약간 묵직하고 남성적? 향이 나네요.. 플라워마켓은 지인이 좋다하여 플로리아와 선물하려합니다. 향수처럼 샘플향을 보내주면 좋을것 같아요' 평점4 22.09.25. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플로리아 / 식물 데코: 안개꽃(핑크) 

134 잘 사용하고 있어요
용기의 고급스러움은 만점주고 싶고
향기는 역시 디퓨저는 어쩔 수 없는지
처음에 엄청 강하고 지금은 발향이 많지 않아요
그래도 은은하게 배어있고
좀 안난다 싶으면 스틱을 뒤집어가면서
관리하니 괜찮아요 평점4 21.08.20. 디퓨저 선택: 브라운 디퓨저 110m

179 우드랜드는 도대체
… 이게 무슨향인지 모르겠고
리틀포레스트는 교보문고 시그니처 향이랑 살짝 비슷한 것 같은데 좀 지린내가;;; 평점4 21.11.02. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 리틀 포레스트 / 식물 데코: 유칼립투스 

180 사무실에 흡연자로 힘들어서
주문해 본 디퓨저 입니당
상쾌한 숲향을 기대하며 주문했는데,
살짝(?)아쉽지만 발향은 좋은것 같아요
스틱 하나만 꽂았는데도 향이 잘 퍼져요 평점4 21.11.01. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 리틀 포레스트 / 식물 데코: 미니 몬스테라 

181 집에 개랑 고양이를 키우다보니 냄새에 민감합니다 ㆍ알콜향이 강하지 않아 좋은데 쟈스민향은 너무 약해서 잘못 고른듯해요 평점4 22.03.14. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 플라워 마켓 / 식물 데코: 미니 몬스테라 

182 전반적으로 만족하고 서비스주신것도 좋아요~
에이프릴후레쉬향은 잘못골랐나? 싶었는데 다음날 되니 향이 은은해지네여~ 잘쓸께여 평점4 21.09.26. 디퓨저 선택: 화이트 디퓨저 200ml / 향: 우드 랜드 / 식물 데코: 몬스테라 

183 향이 나쁘지 않아요 설명서 제대로 안읽고 3개나 꽂아서 첨엔 머리아팠었는데 1개 꽂고 순차적으로
늘려나가시면 되요. 가격 대비 모양도 예쁘고 좋아요. 평점4 22.09.05. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 미니 몬스테라 

184 크기가 작아도 다른 제품에 비해 발향이 좋은것 같아요. 코박고 맡아야 나는 제품들이 많았는데 만족합니다. 조금 더 사용해보고 지속력 좋으면 추가주문 할 예정입니다. 평점4 22.07.19. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 그린 허브 (BEST 3) / 식물 데코: 플라워 L 

185 패키지 예쁘고 3 + 1이라 가성비 좋습니다.
배송 빠르고 포장도 꼼꼼했어요.
생각보다 발향은 안좋은것 같아요..
그래도 은은하네요.. 평점4 21.03.25. 디퓨저

...

1311 마음에 듭니다 향 좋아요 평점4 21.04.28. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 유칼립투스 

1312 향도 좋고 사진과 같아요 평점4 21.02.15. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 목화블렌드 

1313 이쁘고향도무난해서괜춘 평점4 21.02.17. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 우디 포레스트 / 식물 데코: 플라워 L 

1314 발향이 좋아요 빨리 달음 평점4 22.06.04. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 리틀 포레스트 (BEST 2) / 식물 데코: 유칼립투스 

1315 디퓨져 향이 맘에 듭니다 평점4 22.11.11. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 에이프릴 프레쉬(BEST 2) / 식물 데코: 미니 몬스테라 

1316 향이 세지않고 은은해요~ 평점4 21.09.18. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스 

1317 향기가진하고오래갑니다 평점4 21.07.15. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 가든 / 식물 데코: 안개꽃(핑크) 

1318 포장부터 퀄리티가 느껴지고 향은 더욱 좋아
싸구려 느낌없는 향 좋아요 평점4 21.05.25. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 유칼립투스 

1319 향 좋고 배송 꼼꼼합니다 평점4 22.01.11. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 프레쉬 레몬 / 식물 데코: 유칼립투스 

1320 향도 좋고 보기도 이뻐용 평점4 22.01.12. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 안개꽃(핑크) 

1321 개인적으로 스틱2개 추천 평점4 21.11.16. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 안개꽃 (아이보리) 

1322 좋아요 향이 은은합니다. 

1405 향도 굿 인테리어도 굿 평점4 22.08.02. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 안개꽃(핑크) 

1406 집안이 상큼해졌어요^^ 평점4 22.03.22. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 미스티블루 

1407 향기가 좋고 이쁘네요. 평점4 21.04.21. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 로즈 가든 / 식물 데코: 플라워 L 

1408 잘 쓰고 있습니다 ㅌㅋ 평점4 21.10.01. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 미니 몬스테라 

1409 냄새가 좋고 잘퍼져요~ 평점4 21.06.06. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 안개꽃(핑크) 

1410 가성비 좋은 것 같아요 평점4 22.03.15. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 프레쉬 레몬 / 식물 데코: 미니 몬스테라 

1411 넘~~향기가좋아요~~~~~~~ 평점4 22.07.23. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 안개꽃(핑크) 

1412 향이좋아요..만족해요. 평점4 22.09.01. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 미니 몬스테라 

1413 쓰기에좋고향이좋아요 평점4 22.03.02. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 안개꽃(핑크) 

1414 향도 좋고 무난 이뻐요 평점4 22.11.18. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 (BEST 3) / 식물 데코: 유칼립투스 

1415 향 좋아요~많이파세용! 평점4 20.09.08. 디퓨저 선택: 천연 디퓨저 120ml / 향 선택: 가든 스윗피 

1416 향이 엄청 오래가요 굿 평점4 21.09.25. 디퓨저 선택: 브라운 디퓨저 11

페이지 89 

1499 달콤한향이에요 한달되가는데 가까이가면향이나는데
퍼지지는않는거같은느낌에요 ~향은좋아요 평점4 22.02.27. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스 

1500 좋습니다 향도 좋구 평점4 22.06.12. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 미니 몬스테라 

1501 생각보다 넘 작아여 평점4 22.08.11. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 플라워 L 

1502 3+1행사는 좋은데.... 평점4 22.02.20. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 안개꽃(핑크) 

1503 발향 잘되고 좋아요 평점4 21.08.20. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 가든 / 식물 데코: 유칼립투스 

1504 향이 빨리 날라가요 평점4 21.08.29. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 유칼립투스 

1505 좋아여 냄새랑 전부 평점4 22.08.30. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 에이프릴 프레쉬(BEST 2) / 식물 데코: 미니 몬스테라 

1506 발향도 좋아요~🙂 평점4 22.03.26. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플로리아 / 식물 데코: 라그라스(핑크) 

1507 향기가 정말 좋네요 평점4 22.08.29. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 안개꽃(핑크) 

1508 향 진해요 괜찮네요 평점4 21.07.22. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 프레쉬 레몬 / 식물 데코: 유칼립투스 

1509 향기가 강한편에요~ 평점4 22.10.24. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 에이프릴 프레쉬(BEST 2) / 식물 데코: 미니 몬스테라 

1510 얼마나 

페이지 94 

페이지 95 

1585 집에들어가면 라벤더향이 ㅎㅎ 너무좋아요 ㅠㅠ
아쉬운건 전엔 그 장식조화가많앗던거 같은데 선택폭이 적어진게 아쉬울뿐 ㅎ 평점4 22.02.04. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 우드 랜드 / 식물 데코: 플라워 L 

1586 음..너무 향이 달아서 화장실에 놔뒀어요 방에는 너무 쎌거 같아서. 전 달달구리 향은 좋아하질 않아서 ㅎㅎ 평점4 22.03.20. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스 

1587 4가지 향이 모두 괜찮았어요
개인적으로 허브향이 제일 안좋아하는 향이고 나머지는 다 나쁘지 않았어요
재구매의사 있어요 평점4 22.10.04. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 유칼립투스 

1588 풀이 좀 더 파릇파릇했으면 좋았을텐데
그래도 만족해요~ 평점4 20.09.01. 디퓨저 선택: 천연 디퓨저 120ml / 향 선택: 에이프릴 프레쉬 

1589 레몬향이 딱 좋아하는 느낌의 레몬은 아닌데 발향 잘되네요. 욕실에 뒀는데 욕실근처까지 향이나요. 평점4 22.07.06. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 프레쉬 레몬 / 식물 데코: 안개꽃(핑크) 

1590 향도 좋고 발향도 잘 되는편이예요~오래 지속되면 좋겠네요 평점4 22.08.14. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 미니 몬스테라 

1591 선물하려고 주문했는데 배송도 빠르고 포장 따로 하지않아도 선물하기 좋아요~~^^ 평점4 21.04.25. 디퓨저 선택: 화이트 디퓨저 200ml / 향: 가든 스윗 피 / 식물 데코: 스킨답서스 

1592 향은 좋은데 식물이 쭈굴쭈굴?… 해서 좀 아쉬워요 평점4 21.06.06. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 플라워 L 

1593 선물용으로 샀습니다!! 아직 뜯어보지는 

1662 리뷰가좋아서 구매했는데 역시 좋아요 평점4 21.12.29. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 우드 랜드 / 식물 데코: 플라워 S 

1663 향 너무좋고 받침도 있어서 너무 예쁘네요~!!! 평점4 21.01.13. 디퓨저 선택: 화이트 디퓨저 120ml / 향: 플라워 가든 / 식물 데코: 유칼립투스 

1664 발향력도 좋고 배송도 매우 빨랐어요! ㅎㅎ만족합니다 평점4 21.07.17. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 우드 랜드 / 식물 데코: 유칼립투스 

1665 제가 발향이 적은 향을 선택했는지 발향력이 좀 적어요, 담에 발향력 큰걸로 재주문해봐야겠어요 평점4 22.02.25. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 안개꽃 (아이보리) 

1666 빠른배송 감사합니다.
침실에 놔뒀는데 향이 좋네요. 평점4 22.03.29. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 (BEST 3) / 식물 데코: 안개꽃 (아이보리) 

1667 향좋아요 발향력이 강하지는않아요 평점4 21.05.16. 디퓨저 선택: 화이트 디퓨저 200ml / 향: 로즈 가든 / 식물 데코: 아레카야자 

1668 향이 강하지 않고 은은하니 좋아요 ㅎㅎㅎ 평점4 22.08.02. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 라그라스(브라운) 

1669 향이좋아요.
작지만향이강하네요^-^ 평점4 21.07.16. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 클린 코튼 / 식물 데코: 유칼립투스 

1670 향은좋아용ㅎㅎ
나뭇잎은생각보단그냥저냥ㄷㄷㅎ 평점4 20.10.23. 디퓨저 선택: 화이트 천연 디퓨저 120ml / 향 선택: 에이프릴 프레쉬 

1671 향이은은한게 기분좋은향이에요 평점4 21.10.14. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 플라워 L 

1672 향도 괜찮고 디자인도 이쁘

In [12]:
# 4점 리뷰 데이터프레임으로 표현
df2=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df2.head()

,type,category,review,star,date,option
1,헤오드,별점,플라워마켓 우드랜드 리틀포레스트 그린허브 이렇게 4가지향 구매했어요 우드랜드는 남자...,평점5,21.10.22.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 가든 / 식물 데코: 유칼립투스
2,헤오드,별점,후기에 클린코튼향이 좋다는게 많은거같아서 한가지 향으로 통일했는데 향은 은은하니 좋...,평점4,20.12.02.,디퓨저 선택: 리필 용액 200ml / 향: 클린 코튼 / 식물 데코: -
3,헤오드,별점,"우선 아직 다 개봉을 해보진 않았고, 우디 포레스트만 먼저 개봉해 향을 맡아 보았는...",평점4,21.03.17.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 우디 포레스트 / 식물 데코: 유칼립투스
4,헤오드,별점,배송 빠르게 잘받았습니다.\n향은 만족합니다. 데코장식 밑부분이 실리콘으로 감아져있...,평점4,22.02.12.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스
5,헤오드,별점,향 고르기가 어려웠는데 후기 보면서 향을 상상하며 선택했는데 중성적인 향이라 맘에 ...,평점4,21.03.11.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 미니...


In [13]:
# 크롤링한 4점 데이터의 갯수
len(df2)

1715

In [14]:
# 4점 데이터 전처리

df2 = df2.drop_duplicates()
df2 = df2.reset_index(drop=True)
df2_2 = df2['option'].str.split('/',expand=True)
df2 = pd.concat([df2,df2_2],axis=1)
df2.head()

,type,category,review,star,date,option,0,1,2
0,헤오드,별점,플라워마켓 우드랜드 리틀포레스트 그린허브 이렇게 4가지향 구매했어요 우드랜드는 남자...,평점5,21.10.22.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 가든 / 식물 데코: 유칼립투스,디퓨저 선택: 화이트 디퓨저 110ml,향: 플라워 가든,식물 데코: 유칼립투스
1,헤오드,별점,후기에 클린코튼향이 좋다는게 많은거같아서 한가지 향으로 통일했는데 향은 은은하니 좋...,평점4,20.12.02.,디퓨저 선택: 리필 용액 200ml / 향: 클린 코튼 / 식물 데코: -,디퓨저 선택: 리필 용액 200ml,향: 클린 코튼,식물 데코: -
2,헤오드,별점,"우선 아직 다 개봉을 해보진 않았고, 우디 포레스트만 먼저 개봉해 향을 맡아 보았는...",평점4,21.03.17.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 우디 포레스트 / 식물 데코: 유칼립투스,디퓨저 선택: 화이트 디퓨저 110ml,향: 우디 포레스트,식물 데코: 유칼립투스
3,헤오드,별점,배송 빠르게 잘받았습니다.\n향은 만족합니다. 데코장식 밑부분이 실리콘으로 감아져있...,평점4,22.02.12.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스,디퓨저 선택: 화이트 디퓨저 110ml,향: 가든 스윗 피,식물 데코: 유칼립투스
4,헤오드,별점,향 고르기가 어려웠는데 후기 보면서 향을 상상하며 선택했는데 중성적인 향이라 맘에 ...,평점4,21.03.11.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 미니...,디퓨저 선택: 브라운 디퓨저 110ml,향: 블루밍 자스민,식물 데코: 미니 몬스테라


In [15]:
# 전처리 후 크롤링한 4점 데이터의 갯수
len(df2)

1674

In [16]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_4553/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [17]:
# 3점 리뷰 가져오기
d.find_element(By.XPATH,category3).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

1 플라워마켓 우드랜드 리틀포레스트 그린허브 이렇게 4가지향 구매했어요 우드랜드는 남자스킨향 같은데 너무 시원하고 좋은 향이예요 화장실에두니 상쾌하네요 스틱하나만으로 충분해요 개인적으로 제가사용하는 샴푸향과 동일해서 더 맘에 들어요

리틀포레스트는 서재에 어울리는향이예요 방에들어갈때마다 느껴지는 청량감이 좋으네요 화장실보다는 넓어서인지 은은한 발향이 좋아요

플라워마켓은 달달한 향은 아니예요 상품설명처럼 꽃가게 들어갔을때 향과 동일한인데 침대 옆 협탁에 두니 자려고 누웠을때 느껴지는 향이좋아요 단지 침대에 누웠을때만 느껴질정도로 향이 세진 않아요

그린허브는 제가좋아하는 향수와 향이 많이 비슷해요
제일 좋아하는 향이라 애들 침실에 뒀는데 애들이 좋은 향기가 난다고 좋아합니다 단지 발향은 네가지중 제일 약하네요 하루밖에 안되서 인걸까요 ?

발향은 우드랜드 리틀포레스트 플라워마켓 그린허브 순으로 센것 같고요 아무래도 공간크기 영향도 있을것 같네요 거실같은 큰공간은 200미리 두개정도는 필요하지 않을까 싶어요

거실과 주방엔 디퓨저를 놓지않았는데요, 덧붙이자면 발향이 약해서 향이 느껴지지않는다고 해도 거실에서 방으로 들어갔을때의 공기질은 다르긴합니다

길게쓰는거 귀찮아서 안좋아하는데 어제 뜯어보고 너무좋아서 쓰다보니 길어졌네요 그만큼좋고 재구매의사도 있어요~~ 평점5 21.11.18. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스 

2 작은 공방이 환기를 시키고 문닫아 놓으면 새건물인데도 가구냄새와 함께 탁한 공기 냄새? 때문에 불쾌해서 캔들과 디퓨저로 고민하다 주문해봤어요
집에서도 사용하려고 주문하며 향이 궁금해 세가지와 서비스까지 4종 다양하게 골라봤는데 전체 다 코 아프게 강하지 않아 만족하며 가장 좋아하는걸로 공방 한쪽에 놓아봤는데 자주 오던 분이 문 열고 들어서자마자 바로 알아차리고는 너무 좋다고 해주시네요
순간적으로 강하게 뿌려지는게 아닌 은은하게 퍼지는 잔향 같은 느낌이라 공기가 맑아진 기분이라고

40 디퓨져향 고르기 정말 어렵네요 구매한
세가지향중 플라워마켓이 제일 괜찮은데
리드 두개 꽂았더니 향이 너무 진해서 한개뺐는데 그래도 진하네요~ 평점3 22.03.05. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 라그라스(핑크) 

페이지 3 

41 디퓨져 향기는 블루밍자스민이 제일 좋네요
에이프릴프레쉬만 제 취향이 아니고 나머지 3가지는 다 괜찮습니다
같이 꼽는 식물중에는 무료로 받은 미니 몬스테라가 가장 괜찮았구요…제가 운이 없어서인지 그림과 달리 플라워L은 잎이 작은 게 걸린 것 같아서 사진 만큼 안예쁘네요
그리고 라탄 품절되서 못산게 아쉽습니다
정말 많이많이 아쉬운 점은 말린꽃 플라워s에서 안좋은 냄새가 나고(아까워서 이건 꼽았습니다만..)
미스티블루에서는 정말 찌린내가 심하게 나서 버렸습니다
보내신 분은 한번 맡아보세요 ㅠㅠ 평점3 21.10.01. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 스프링 데이 / 식물 데코: 플라워 L 

42 발향이나 향자체는 나쁘지않구요
그저 신경쓰이던게 사이즈정도인데 비교하자면
대충 담배곽 이면 주둥이?부분빼고 다가려질정도니
참고하세여 평점3 22.08.31. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 (BEST 3) / 식물 데코: 미니 몬스테라 

43 가든스윗피 평이좋아서 겟했는데 제스탈은 아니더라구요ㅠㅠ 같이산 라벤더가 훨씬좋아요~
발향은 정말잘되요~ 스틱3개꽂았다가 하나뺐어요~ 평점3 21.04.22. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 플라워 L 

44 처음엔 향이 느껴질정도로 났는데 스틱을 바꿔끼워줘도 시간이 지날수록 향이 잘 안나는것같아서 좀 아쉬웟어요... 너무 강해서 머리아프고 그런 향이 아니라 은은해서 좋았는데 유지가 잘 안되는것 같아서 아쉽네요 ㅠ 평점3 21.02.14. 디퓨저 선택: 브라운 디퓨저 120ml / 향: 하와이안 라임 비치 / 식물 데코: 유칼립투스 

45 조금 아쉬워요~리뷰

107 불랙체리는 갠적으로 넘 다네요 ㅎㅎ 청포도 쮸쮸바 같아용 ㅋ 평점3 20.10.08. 디퓨저 선택: 화이트 천연 디퓨저 120ml / 향 선택: 블랙 체리 

108 재구매인데요. 뚜껑 따다가 손에 약간 상처났어요. 물론 제가 잘 못해서 그런 것도 있지만요. 향은 좋아요. 다음에도 재구매 의사는 있어요. 평점3 21.09.27. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 스프링 데이 / 식물 데코: 유칼립투스 

109 선물용으로 샀는데
타사제품보다 포장 박스 재질이나
쇼핑백이 좀 싼티 나네요ㅜ
선물용으로 구매 하실분들은 참고
하셔요. 향은 선물용 이라 오픈하지 못해 모르겠어요 평점3 21.01.29. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 플라워 S 

110 처음에는 향이 좀 진했는데 시간이 지날수록 은은하니 좋아요 평점3 20.11.19. 디퓨저 선택: 화이트 천연 디퓨저 120ml / 향 선택: 가든 스윗피 

111 화장실엔 블랙체리가 좋아요.
다음엔 이 향만 주문해야겠어요. 평점3 22.06.03. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 미니 몬스테라 

112 베스트향이라고 구매했는데 저는안맞는듯해요~너무 진하네여 평점3 22.06.24. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 유칼립투스 

113 조금 더 향이 잘 발향되었으면 좋겠어요.
향은 좋은데 막 흔들고 스틱 꺼내서 공중에 쫌 뿌리고하면 향이 나는것같아요.
그래도 향은 좋아요 평점3 22.06.14. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플로리아 / 식물 데코: 유칼립투스 

114 방에 두면 방 전체적인 향기가 좋아진다는 그런 그라마틱한 효과는 없고요 그냥 그 주변만 향기나고 현관 같은 밀폐된 공간에 두면 좋아요 평점3 21.02.25. 디퓨저 선택: 화이트 디퓨저 120ml / 향: 미스티 포레스트 / 식물 데코: 유칼립투스 

11

페이지 13 

184 힐링라벤더는 생각보다 제 취향이 아니라서 다 쓰고나서 리필만 사서 다른향으로 바꿨어요~ 평점3 22.08.14. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 프레쉬 레몬 / 식물 데코: 미니 몬스테라 

185 배송은 빨랐지만, 제가 생각한 로즈향, 꽃집냄새 느낌은 아니라서 아쉬워요,,,, 평점3 21.05.05. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 가든 / 식물 데코: 유칼립투스 

186 생각보다 향이 크게나질 않아요 가까이 가야 나고 ㅎ 넘 진한거보단 낫지만요 평점3 21.07.02. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 안개꽃(핑크) 

187 좀더 진한 향을 원했는데 은은해서 살짝 아쉬웠어요
그건 개인취향이니까요 평점3 21.04.10. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스 

188 향이 은은하니 좋아요 인테리어 효과도 있네요 담에 큰걸루 구매하고싶어요 평점3 21.02.20. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스 

189 오래 향이 난다라는 글 보고 구매했어요
초반에는 향이 잘 나다가 어느정도 지나서 위치도 바꿔보고 스틱도 교체해봤는데 향이 많이 안나요 ㅜㅜ 평점3 21.06.21. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 미니 몬스테라 

190 우드랜드 향은 생각했던거랑 좀 다르네요ㅠ 나프탈렌 향인것 같기도 하고 저는 불호인데ㅠ 향은 잘 퍼져요 평점3 21.08.12. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 우드 랜드 / 식물 데코: 안개꽃 (아이보리) 

191 스틱이 한개 덜 와서 별뺐어요. 보내달라기 민망해서 그냥 구매확정합니다 평점3 22.06.28. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 플라워 마켓 (BEST 1) / 식물 데코: 안개꽃 (아이보리) 

192 향은 너무 좋은데 제가 예민한 편

258 향은 좋구요 스킨답서스는 조화티가 너무 많이나요 ㅠㅠ 평점3 21.05.18. 디퓨저 선택: 화이트 디퓨저 200ml / 향: 에이프릴 프레쉬 / 식물 데코: 스킨답서스 

259 어디가 잘안닫힌건지 제대로 꼭 닫아서 배송해주세요ㅔ 평점3 21.07.12. 디퓨저 선택: 리필 용액 110ml / 향: 플라워 가든 / 식물 데코: 식물 미포함 

260 기대한것보단 별로였어여
잎도 사진보단 저렴한 느낌^^;
디퓨저액이 약간 노란? 연두?빛나요 평점3 21.08.06. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 플라워 가든 / 식물 데코: 유칼립투스 

261 리필 뚜껑은 요긴해 보임. 아직 안 뜯어봐서 향은 모름. 평점3 21.02.09. 디퓨저 선택: 리필 용액 110ml / 향: 프레쉬 레몬 / 식물 데코: - 

262 어느 카페에서 보고 구매했는데..
가격대비 만족합니다. 평점3 22.08.26. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 우드 랜드 / 식물 데코: 안개꽃(핑크) 

263 선물하려고 12개 주문했습니다.
향은 많은 분들이 좋다고해주신 클린 코튼 향으로 선택했습니다!~~

배송과정에 문제가 있었으나
잘처리해주셔서 해결되었습니다. 평점3 20.12.27. 디퓨저 선택: 브라운 디퓨저 120ml / 향: 클린 코튼 / 식물 데코: 유칼립투스 

페이지 17 

페이지 18 

페이지 19 

264 제가 예민한 편이라 눈이 시려요...
향은 넘 좋아요~ 평점3 21.04.01. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 스프링 데이 / 식물 데코: 안개꽃(핑크) 

265 향이 너무 없어요
화장실에 놓아뒀는데 즤집만 이런걸까요
다른 진한향 화장실에 어울리는건 어떤건지 알고싶네요. ㅜㅜ 평점3 22.06.14. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 유칼립투스 

266 향이 기대했던 거랑 좀 차이가 있지만
잘쓸께요 평점3 21.05.08. 디퓨저 선택: 브라운 디퓨저 110ml / 향

페이지 23 

344 만족합니다~~ㅎㅎㅋ 평점3 20.07.17. 디퓨저 선택: 천연 디퓨저 120ml / 향 선택: 하와이안 라임 비치 

345 발향이 잘 안나요ㅠ 평점3 21.08.24. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 플라워 L 

346 역시 블랙체리 💜 평점3 21.07.03. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 유칼립투스 

347 가성비 비해 좋아요 평점3 21.08.28. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 스프링 데이 / 식물 데코: 안개꽃 (아이보리) 

348 인위적인 향이긴 하네요.. 향이 강해서 디퓨저스틱 2개만 꽂았어요 :( 평점3 20.11.09. 디퓨저 선택: 리필 용액 200ml / 향 선택: 미스티 포레스트 

349 좋아요.좋습니다 .. 평점3 21.12.14. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 미니 몬스테라 

350 예쁜데 양이
작아요 평점3 21.06.16. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 유칼립투스 

351 처음엔 발향이 진짜강했는데 시간이 지나면서 확실히 발향이 좀약해지는 느낌은있어요 평점3 21.05.23. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 유칼립투스 

352 괜찮은거같아요 :) 평점3 21.01.02. 디퓨저 선택: 브라운 디퓨저 120ml / 향: 클린 코튼 / 식물 데코: 유칼립투스 

353 좋습니다. 좋아요. 평점3 21.12.14. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스 

354 발향이 잘되네요~~ 평점3 21.07.12. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스 

355 100이 양이 작네요 평점3 21.08.03. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브

페이지 27 

424 재주문해서 사용하는데 앞으로도 또 사용하고 싶네요 평점3 21.07.21. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 프레쉬 레몬 / 식물 데코: 유칼립투스 

425 좋아요 보기에 좋아요 평점3 22.06.13. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 미니 몬스테라 

426 좋아요 좋아요 ~~~~~~~ 평점3 21.10.15. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 유칼립투스 

427 좋아요좋아요좋아요. 평점3 20.12.30. 디퓨저 선택: 화이트 디퓨저 120ml / 향: 클린 코튼 / 식물 데코: 플라워 L 

428 머리아픔 머리아픔 머리아픔 평점3 21.03.03. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 미니 몬스테라 

429 좋아요좋아요좋아요좋아요좋아요좋아요 평점3 21.01.08. 디퓨저 선택: 화이트 디퓨저 120ml / 향: 가든 스윗 피 / 식물 데코: 미니 몬스테라 



In [18]:
# 3점 리뷰 데이터프레임으로 표현
df3=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df3.head()

,type,category,review,star,date,option
1,헤오드,별점,플라워마켓 우드랜드 리틀포레스트 그린허브 이렇게 4가지향 구매했어요 우드랜드는 남자...,평점5,21.11.18.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스
2,헤오드,별점,작은 공방이 환기를 시키고 문닫아 놓으면 새건물인데도 가구냄새와 함께 탁한 공기 냄...,평점5,22.07.08.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스
3,헤오드,별점,상세페이지 설명처럼 발향이 확실하고 기존에 쓰던것 보다 진하다는 느낌을 확실히 받았...,평점3,21.11.02.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 안...
4,헤오드,별점,화장실에서 쓰려고 골랐는데 냄새 너무 진해서 머리 아프고 약간 꿀향? 비슷한게 있어...,평점3,21.08.31.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스
5,헤오드,별점,이벤트 중일 때 저렴하게 구매했어요.\n클린 코튼이 생각했던 향은 아니었지만 은은하...,평점3,20.11.12.,디퓨저 선택: 천연 디퓨저 120ml / 향 선택: 클린 코튼


In [19]:
# 전처리 후 크롤링한 3점 데이터의 갯수
len(df3)

429

In [20]:
# 3점 데이터 전처리

df3 = df3.drop_duplicates()
df3 = df3.reset_index(drop=True)
df3_3 = df3['option'].str.split('/',expand=True)
df3 = pd.concat([df3,df3_3],axis=1)
df3.head()

,type,category,review,star,date,option,0,1,2
0,헤오드,별점,플라워마켓 우드랜드 리틀포레스트 그린허브 이렇게 4가지향 구매했어요 우드랜드는 남자...,평점5,21.11.18.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스,디퓨저 선택: 브라운 디퓨저 110ml,향: 그린 허브,식물 데코: 유칼립투스
1,헤오드,별점,작은 공방이 환기를 시키고 문닫아 놓으면 새건물인데도 가구냄새와 함께 탁한 공기 냄...,평점5,22.07.08.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스,디퓨저 선택: 화이트 디퓨저 110ml,향: 가든 스윗 피,식물 데코: 유칼립투스
2,헤오드,별점,상세페이지 설명처럼 발향이 확실하고 기존에 쓰던것 보다 진하다는 느낌을 확실히 받았...,평점3,21.11.02.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 안...,디퓨저 선택: 브라운 디퓨저 110ml,향: 에이프릴 프레쉬,식물 데코: 안개꽃 (아이보리)
3,헤오드,별점,화장실에서 쓰려고 골랐는데 냄새 너무 진해서 머리 아프고 약간 꿀향? 비슷한게 있어...,평점3,21.08.31.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스,디퓨저 선택: 화이트 디퓨저 110ml,향: 그린 허브,식물 데코: 유칼립투스
4,헤오드,별점,이벤트 중일 때 저렴하게 구매했어요.\n클린 코튼이 생각했던 향은 아니었지만 은은하...,평점3,20.11.12.,디퓨저 선택: 천연 디퓨저 120ml / 향 선택: 클린 코튼,디퓨저 선택: 천연 디퓨저 120ml,향 선택: 클린 코튼,None


In [21]:
# 전처리 후 크롤링한 3점 데이터의 갯수
len(df3)

410

In [22]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_4553/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [23]:
# 2점 리뷰 가져오기
d.find_element(By.XPATH,category4).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

페이지 2 

1 인터넷에 수많은 디퓨져가 있고, 향을 되게 중요하게 생각하는지라 향수같은건 무조건 오프라인으로만 구매하는데, 코로나 때문에 시향이 안되는 곳도 많아서 인터넷으로 처음 디퓨저를 구매했습니다. 후기가 좋아 여기서 구매 했는데 저희 솔직한 후기 써볼게요.

디퓨저 치고 발향력도 좋고 병도 이쁘고 다 맘에 듭니다. 다만 역시 시향을 못하고 사니 문제가 있네요.

가든레시피는 양키캔들 향이라 알고 구입했고, 그것과 비슷하고 좋습니다.
다만 블랙체리는 업체마다 향이 달라서 그냥 구매 해봤는데, 노래방? 같은 데서 나는 향이 납니다. 정말 비추에요. 이건 뭐 판매자님 잘못은 아니고, 향은 주관적인거라 제기준 드리는 말씀입니다.

이사를 한 터라 다른 곳에서도 디퓨저 또 구매했습니다. 거기도 블랙체리 구매했으니, 한달 사용후기 올릴 수 있으면 비교해서 올리겠습니다. 평점2 21.02.25. 디퓨저 선택: 화이트 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 유칼립투스 

2 집에 반려동물이 있어 특유의 체취가 있어 인체 무해한 디퓨저 검색하다 구매해봤어요. 화장실과 거실에 각 놔뒀습니다. 지속력이 좋으면 좋겠어요.


- 며칠 지나니 향이 디퓨저에 코를 대고 맡아야 날 정도로 발향력이 떨어지네요. 스틱 2개로 시작했는데 다음날이 제일 향이 강했고 그 뿐. 은은함이 아니라 향이 안 퍼져요. 거실(35평)에 각 다른 자리에 (문 여닫는 곳이라 공기 흐름 좋음) 2개 놔뒀습니다만, 외출했다 들어와도 향은 없고.. 뭐 그래요. 평점2 22.05.31. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 안개꽃 (아이보리) 

3 포장 꼼꼼하게 잘 왔고 첫날이라 발향은 좋아요. 설명서대로 사용해보고 효과 만족하면 재구매하려합니다.


- 며칠 후 : 화장실에 놔두고 며칠 지나니 향이 안 나네요. 좁은 공간이고 수시로 사람이 드나드는 곳인데…발향이 이렇게 금세 멈추다니ㅠㅠ 좁아서 처음 하나만 꽂았었으나 오늘 스틱 하나 더

In [24]:
# 2점 리뷰 데이터프레임으로 표현
df4=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df4.head()

,type,category,review,star,date,option
1,헤오드,별점,"인터넷에 수많은 디퓨져가 있고, 향을 되게 중요하게 생각하는지라 향수같은건 무조건 ...",평점5,21.02.25.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 유칼립투스
2,헤오드,별점,집에 반려동물이 있어 특유의 체취가 있어 인체 무해한 디퓨저 검색하다 구매해봤어요....,평점2,22.05.31.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 안개꽃...
3,헤오드,별점,포장 꼼꼼하게 잘 왔고 첫날이라 발향은 좋아요. 설명서대로 사용해보고 효과 만족하면...,평점2,22.05.31.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 안...
4,헤오드,별점,받침이랑 놓으니 깔끔하고 이쁘긴한데 향은 도대체가 나질 않아요 코를 갖다 킁킁거려야...,평점2,21.11.13.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 플라워 L
5,헤오드,별점,처음이라 그런가 하나만 꽂아도 향이 잘 나요. 근데 데코 조화에서 썩은냄새 나서 별...,평점2,20.08.20.,디퓨저 선택: 천연 디퓨저 120ml / 향 선택: 힐링 라벤더


In [25]:
# 크롤링한 2점 데이터의 갯수
len(df4)

41

In [26]:
# 2점 데이터 전처리

df4 = df4.drop_duplicates()
df4 = df4.reset_index(drop=True)
df4_4 = df4['option'].str.split('/',expand=True)
df4 = pd.concat([df4,df4_4],axis=1)
df4.head()

,type,category,review,star,date,option,0,1,2
0,헤오드,별점,"인터넷에 수많은 디퓨져가 있고, 향을 되게 중요하게 생각하는지라 향수같은건 무조건 ...",평점5,21.02.25.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 블랙 체리 / 식물 데코: 유칼립투스,디퓨저 선택: 화이트 디퓨저 110ml,향: 블랙 체리,식물 데코: 유칼립투스
1,헤오드,별점,집에 반려동물이 있어 특유의 체취가 있어 인체 무해한 디퓨저 검색하다 구매해봤어요....,평점2,22.05.31.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 힐링 라벤더 / 식물 데코: 안개꽃...,디퓨저 선택: 브라운 디퓨저 110ml,향: 힐링 라벤더,식물 데코: 안개꽃 (아이보리)
2,헤오드,별점,포장 꼼꼼하게 잘 왔고 첫날이라 발향은 좋아요. 설명서대로 사용해보고 효과 만족하면...,평점2,22.05.31.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 에이프릴 프레쉬 / 식물 데코: 안...,디퓨저 선택: 브라운 디퓨저 110ml,향: 에이프릴 프레쉬,식물 데코: 안개꽃 (아이보리)
3,헤오드,별점,받침이랑 놓으니 깔끔하고 이쁘긴한데 향은 도대체가 나질 않아요 코를 갖다 킁킁거려야...,평점2,21.11.13.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 블루밍 자스민 / 식물 데코: 플라워 L,디퓨저 선택: 화이트 디퓨저 110ml,향: 블루밍 자스민,식물 데코: 플라워 L
4,헤오드,별점,처음이라 그런가 하나만 꽂아도 향이 잘 나요. 근데 데코 조화에서 썩은냄새 나서 별...,평점2,20.08.20.,디퓨저 선택: 천연 디퓨저 120ml / 향 선택: 힐링 라벤더,디퓨저 선택: 천연 디퓨저 120ml,향 선택: 힐링 라벤더,None


In [27]:
# 전처리 후 크롤링한 2점 데이터의 갯수
len(df4)

40

In [28]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_4553/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [29]:
# 1점 리뷰 가져오기
d.find_element(By.XPATH,category5).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

페이지 2 

1 1. 아쉬운 점

탑노트가 너무 강해서인지는 모르겠지만 쉽게 표현하자면 치과냄새가 너무, 너무너무너무 심합니다. 조향을 과하게 하신건지 재료가 별로인건지, 아무리 시트러스 계열이라 해도 특유의 시원한 향이 아닌, 저렴한 느낌이 팍 옵니다. 시향을 할 수록 머리가 너무 아프고 너무 느끼하고 거북해서 우디향은 바로 다 버렸습니다.. 싼게 비지떡인걸까요.. 우디향이 안맞기는 이곳이 처음이였습니다. 많이 아쉽습니다.

2. 좋은 점

디퓨저를 놓는 주 목적은 향이나 조화를 꽂아 두므로써 인테리어 역할도 한다고 봅니다. 이 가격에 조화를 이렇게 주는건 만족스러웠습니다.

*사진 속 조화는 해당브랜드 것이며 디퓨저는 타브랜드 것으로 꽂아 둔 것입니다. 평점1 21.04.19. 디퓨저 선택: 화이트 디퓨저 200ml / 향: 우디 포레스트 / 식물 데코: 스킨답서스 

2 발향이 잘 된다는 분들 리뷰 보고 구매한건데 진짜 진심 발향 1도 안되요 집에 들어왔을 때 디퓨저 향기가 아예 코 끝에 스쳐 지나가지도 않습니다.. 심지어 스틱 4개를 다 꽂았고 몇일 뒤에 뒤집어 꽂았는데도 발향이 안되는데 상품이 불량인건지 원래 이런건지; 이거 쓰기 전에 다른 타 제품 사용했을 땐 집에 들어오자마자 향기가 너무 진해서 현관에서 멀리 떨어진 곳에 둘 정도였는데 이건.. 코 박고 향기 맡아야하니.. 전 다신 안살듯합니다 평점1 21.12.23. 디퓨저 선택: 브라운 디퓨저 110ml / 향: 우드 랜드 / 식물 데코: 유칼립투스 

페이지 3 

3 1. 아쉬운 점

탑노트가 너무 강해서인지는 모르겠지만 쉽게 표현하자면 치과냄새가 너무, 너무너무너무 심합니다. 조향을 과하게 하신건지 재료가 별로인건지, 아무리 시트러스 계열이라 해도 특유의 시원한 향이 아닌, 저렴한 느낌이 팍 옵니다. 시향을 할 수록 머리가 너무 아프고 너무 느끼하고 거북해서 우디향은 바로 다 버렸습니다.. 싼게 비지떡인걸까요.. 우디향이 안맞기는 이곳이 처음이였습니다. 많이 아쉽습니다.



In [30]:
# 1점 리뷰 데이터프레임으로 표현
df5=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df5.head()

,type,category,review,star,date,option
1,헤오드,별점,1. 아쉬운 점\n\n탑노트가 너무 강해서인지는 모르겠지만 쉽게 표현하자면 치과냄새...,평점5,21.04.19.,디퓨저 선택: 화이트 디퓨저 200ml / 향: 우디 포레스트 / 식물 데코: 스킨답서스
2,헤오드,별점,발향이 잘 된다는 분들 리뷰 보고 구매한건데 진짜 진심 발향 1도 안되요 집에 들어...,평점1,21.12.23.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 우드 랜드 / 식물 데코: 유칼립투스
3,헤오드,별점,너무 황당해서 다시 댓글 달아요 . 저는 댓글에 사은품 얘기 한적도 없고 본품이 향...,평점1,21.02.28.,디퓨저 선택: 화이트 디퓨저 200ml / 향: 우디 포레스트 / 식물 데코: 스킨답서스
4,헤오드,별점,1. 아쉬운 점\n\n탑노트가 너무 강해서인지는 모르겠지만 쉽게 표현하자면 치과냄새...,평점1,21.04.19.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 플라워 가든 / 식물 데코: 플라워 T


In [31]:
# 전처리 후 크롤링한 1점 데이터의 갯수
len(df5)

4

In [32]:
# 1점 데이터 전처리

df5 = df5.drop_duplicates()
df5 = df5.reset_index(drop=True)
df5_5 = df5['option'].str.split('/',expand=True)
df5 = pd.concat([df5,df5_5],axis=1)
df5.head()

,type,category,review,star,date,option,0,1,2
0,헤오드,별점,1. 아쉬운 점\n\n탑노트가 너무 강해서인지는 모르겠지만 쉽게 표현하자면 치과냄새...,평점5,21.04.19.,디퓨저 선택: 화이트 디퓨저 200ml / 향: 우디 포레스트 / 식물 데코: 스킨답서스,디퓨저 선택: 화이트 디퓨저 200ml,향: 우디 포레스트,식물 데코: 스킨답서스
1,헤오드,별점,발향이 잘 된다는 분들 리뷰 보고 구매한건데 진짜 진심 발향 1도 안되요 집에 들어...,평점1,21.12.23.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 우드 랜드 / 식물 데코: 유칼립투스,디퓨저 선택: 브라운 디퓨저 110ml,향: 우드 랜드,식물 데코: 유칼립투스
2,헤오드,별점,너무 황당해서 다시 댓글 달아요 . 저는 댓글에 사은품 얘기 한적도 없고 본품이 향...,평점1,21.02.28.,디퓨저 선택: 화이트 디퓨저 200ml / 향: 우디 포레스트 / 식물 데코: 스킨답서스,디퓨저 선택: 화이트 디퓨저 200ml,향: 우디 포레스트,식물 데코: 스킨답서스
3,헤오드,별점,1. 아쉬운 점\n\n탑노트가 너무 강해서인지는 모르겠지만 쉽게 표현하자면 치과냄새...,평점1,21.04.19.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 플라워 가든 / 식물 데코: 플라워 T,디퓨저 선택: 브라운 디퓨저 110ml,향: 플라워 가든,식물 데코: 플라워 T


In [33]:
# 전처리 후 크롤링한 1점 데이터의 갯수
len(df5)

4

In [34]:
# 5점~1점 데이터합치기
df_ascent = pd.concat([df1,df2,df3,df4,df5])
df_ascent.head()

,type,category,review,star,date,option,0,1,2
0,헤오드,별점,플라워마켓 우드랜드 리틀포레스트 그린허브 이렇게 4가지향 구매했어요 우드랜드는 남자...,평점5,21.11.18.,디퓨저 선택: 브라운 디퓨저 110ml / 향: 그린 허브 / 식물 데코: 유칼립투스,디퓨저 선택: 브라운 디퓨저 110ml,향: 그린 허브,식물 데코: 유칼립투스
1,헤오드,별점,작은 공방이 환기를 시키고 문닫아 놓으면 새건물인데도 가구냄새와 함께 탁한 공기 냄...,평점5,22.07.08.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 가든 스윗 피 / 식물 데코: 유칼립투스,디퓨저 선택: 화이트 디퓨저 110ml,향: 가든 스윗 피,식물 데코: 유칼립투스
2,헤오드,별점,향에 민감한 편이라 디퓨저를 구매할 때 후기 몇십개씩 보고 사는데\n가격도 너무 괜...,평점5,21.01.04.,디퓨저 선택: 화이트 디퓨저 120ml / 향: 그린 허브 / 식물 데코: 플라워 L,디퓨저 선택: 화이트 디퓨저 120ml,향: 그린 허브,식물 데코: 플라워 L
3,헤오드,별점,"플로리아, 라그라스 우왕! 매우 퀸 queen 처럼 우아하고 아름다우며 너무 너무 ...",평점5,22.10.04.,디퓨저 선택: 화이트 디퓨저 110ml / 향: 플로리아 / 식물 데코: 플라워 T,디퓨저 선택: 화이트 디퓨저 110ml,향: 플로리아,식물 데코: 플라워 T
4,헤오드,별점,오랜만에 다시 재구매했는데 그동안 많은 발전이 있었네요☺️\n센스가 어찌 이리 좋은...,평점5,21.08.12.,디퓨저 선택: 화이트 디퓨저 200ml / 향: 클린 코튼 / 식물 데코: 카라(핑...,디퓨저 선택: 화이트 디퓨저 200ml,향: 클린 코튼,식물 데코: 카라(핑크) 2ea


In [35]:
# 1점~5점 데이터의 총 갯수
len(df_ascent)

3673

In [36]:
### CSV파일로 저장
df_ascent.to_csv('헤오드_평점별리뷰.csv')